In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [14]:
from src.models.nns import (
    MultimodalEncoder,
    SetEncoder,
    ConcatEncoder
)
import torch

In [18]:
input_size = 40
output_size = 40
hidden_units = [40]
batch_size = 32
n_modalities = 3

xs = [torch.randn(batch_size, input_size) for _ in range(n_modalities)]
xs_missing = [None, torch.randn(batch_size, input_size), None]

In [12]:
set_encoder = SetEncoder(
    input_size, output_size, hidden_units=hidden_units
)

In [19]:
output = set_encoder(xs_missing)

In [20]:
output.shape

torch.Size([32, 40])

In [23]:
concat_encoder = ConcatEncoder(
    n_modalities, input_size, output_size, hidden_units=hidden_units
)

In [24]:
concat_encoder

ConcatEncoder(
  (0): LambdaLayer()
  (1): Linear(in_features=120, out_features=40, bias=True)
  (2): ReLU()
  (3): Linear(in_features=40, out_features=40, bias=True)
)

In [28]:
output = concat_encoder(xs_missing)

In [30]:
output.shape

torch.Size([32, 40])

In [ ]:
encoder = MultimodalEncoder()